In [1]:
from __future__ import division
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import brown
import math
import numpy as np
import sys

In [3]:
nltk.download('all-corpora')
nltk.download('punkt')

[nltk_data] Downloading collection 'all-corpora'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to C:\Users\Milind.DESKTOP-
[nltk_data]    |     GBR1BS2\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Milind.DESKTOP-
[nltk_data]    |     GBR1BS2\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\Milind.DESKTOP-
[nltk_data]    |     GBR1BS2\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to C:\Users\Milind.DESKTOP-
[nltk_data]    |     GBR1BS2\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\Milind.DESKTOP-
[nltk_data]    |     GBR1BS2\AppData\

[nltk_data]    | Downloading package omw to C:\Users\Milind.DESKTOP-
[nltk_data]    |     GBR1BS2\AppData\Roaming\nltk_data...
[nltk_data]    |   Package omw is already up-to-date!
[nltk_data]    | Downloading package omw-1.4 to
[nltk_data]    |     C:\Users\Milind.DESKTOP-
[nltk_data]    |     GBR1BS2\AppData\Roaming\nltk_data...
[nltk_data]    |   Package omw-1.4 is already up-to-date!
[nltk_data]    | Downloading package opinion_lexicon to
[nltk_data]    |     C:\Users\Milind.DESKTOP-
[nltk_data]    |     GBR1BS2\AppData\Roaming\nltk_data...
[nltk_data]    |   Package opinion_lexicon is already up-to-date!
[nltk_data]    | Downloading package panlex_swadesh to
[nltk_data]    |     C:\Users\Milind.DESKTOP-
[nltk_data]    |     GBR1BS2\AppData\Roaming\nltk_data...
[nltk_data]    |   Package panlex_swadesh is already up-to-date!
[nltk_data]    | Downloading package paradigms to
[nltk_data]    |     C:\Users\Milind.DESKTOP-
[nltk_data]    |     GBR1BS2\AppData\Roaming\nltk_data...
[nltk

[nltk_data]    |   Package webtext is already up-to-date!
[nltk_data]    | Downloading package wordnet to
[nltk_data]    |     C:\Users\Milind.DESKTOP-
[nltk_data]    |     GBR1BS2\AppData\Roaming\nltk_data...
[nltk_data]    |   Package wordnet is already up-to-date!
[nltk_data]    | Downloading package wordnet2021 to
[nltk_data]    |     C:\Users\Milind.DESKTOP-
[nltk_data]    |     GBR1BS2\AppData\Roaming\nltk_data...
[nltk_data]    |   Package wordnet2021 is already up-to-date!
[nltk_data]    | Downloading package wordnet31 to
[nltk_data]    |     C:\Users\Milind.DESKTOP-
[nltk_data]    |     GBR1BS2\AppData\Roaming\nltk_data...
[nltk_data]    |   Package wordnet31 is already up-to-date!
[nltk_data]    | Downloading package wordnet_ic to
[nltk_data]    |     C:\Users\Milind.DESKTOP-
[nltk_data]    |     GBR1BS2\AppData\Roaming\nltk_data...
[nltk_data]    |   Package wordnet_ic is already up-to-date!
[nltk_data]    | Downloading package words to C:\Users\Milind.DESKTOP-
[nltk_data]  

True

In [2]:
# Parameters to the algorithm. Currently set to values that was reported
# in the paper to produce "best" results.
ALPHA = 0.2
BETA = 0.45
ETA = 0.4
PHI = 0.2
DELTA = 0.85

In [3]:
brown_freqs = dict()
N = 0

######################## word similarity ##########################

In [4]:
def get_best_synset_pair(word_1, word_2):
    """ 
    Choose the pair with highest path similarity among all pairs. 
    Mimics pattern-seeking behavior of humans.
    """
    max_sim = -1.0
    synsets_1 = wn.synsets(word_1)
    synsets_2 = wn.synsets(word_2)
    if len(synsets_1) == 0 or len(synsets_2) == 0:
        return None, None
    else:
        max_sim = -1.0
        best_pair = None, None
        for synset_1 in synsets_1:
            for synset_2 in synsets_2:
                sim = wn.path_similarity(synset_1, synset_2)
                if sim == None:
                    sim = 0.
                if sim > max_sim:
                    max_sim = sim
                    best_pair = synset_1, synset_2
        return best_pair

In [5]:
def length_dist(synset_1, synset_2):
    """
    Return a measure of the length of the shortest path in the semantic 
    ontology (Wordnet in our case as well as the paper's) between two 
    synsets.
    """
    l_dist = sys.maxsize
    if synset_1 is None or synset_2 is None: 
        return 0.0
    if synset_1 == synset_2:
        # if synset_1 and synset_2 are the same synset return 0
        l_dist = 0.0
    else:
        wset_1 = set([str(x.name()) for x in synset_1.lemmas()])        
        wset_2 = set([str(x.name()) for x in synset_2.lemmas()])
        if len(wset_1.intersection(wset_2)) > 0:
            # if synset_1 != synset_2 but there is word overlap, return 1.0
            l_dist = 1.0
        else:
            # just compute the shortest path between the two
            l_dist = synset_1.shortest_path_distance(synset_2)
            if l_dist is None:
                l_dist = 0.0
    # normalize path length to the range [0,1]
    return math.exp(-ALPHA * l_dist)

In [6]:
def hierarchy_dist(synset_1, synset_2):
    """
    Return a measure of depth in the ontology to model the fact that 
    nodes closer to the root are broader and have less semantic similarity
    than nodes further away from the root.
    """
    h_dist = sys.maxsize
    if synset_1 is None or synset_2 is None: 
        return h_dist
    if synset_1 == synset_2:
        # return the depth of one of synset_1 or synset_2
        h_dist = max([x[1] for x in synset_1.hypernym_distances()])
    else:
        # find the max depth of least common subsumer
        hypernyms_1 = {x[0]:x[1] for x in synset_1.hypernym_distances()}
        hypernyms_2 = {x[0]:x[1] for x in synset_2.hypernym_distances()}
        lcs_candidates = set(hypernyms_1.keys()).intersection(
            set(hypernyms_2.keys()))
        if len(lcs_candidates) > 0:
            lcs_dists = []
            for lcs_candidate in lcs_candidates:
                lcs_d1 = 0
                if lcs_candidate in hypernyms_1 :
                    lcs_d1 = hypernyms_1[lcs_candidate]
                lcs_d2 = 0
                if lcs_candidate in hypernyms_2:
                    lcs_d2 = hypernyms_2[lcs_candidate]
                lcs_dists.append(max([lcs_d1, lcs_d2]))
            h_dist = max(lcs_dists)
        else:
            h_dist = 0
    return ((math.exp(BETA * h_dist) - math.exp(-BETA * h_dist)) / 
        (math.exp(BETA * h_dist) + math.exp(-BETA * h_dist)))


In [7]:
def word_similarity(word_1, word_2):
    synset_pair = get_best_synset_pair(word_1, word_2)
    return (length_dist(synset_pair[0], synset_pair[1]) * 
        hierarchy_dist(synset_pair[0], synset_pair[1]))

######################### sentence similarity ##########################

In [8]:
def most_similar_word(word, word_set):
    """
    Find the word in the joint word set that is most similar to the word
    passed in. We use the algorithm above to compute word similarity between
    the word and each word in the joint word set, and return the most similar
    word and the actual similarity value.
    """
    max_sim = -1.0
    sim_word = ""
    for ref_word in word_set:
      sim = word_similarity(word, ref_word)
      if sim > max_sim:
          max_sim = sim
          sim_word = ref_word
    return sim_word, max_sim

In [9]:
def info_content(lookup_word):
    """
    Uses the Brown corpus available in NLTK to calculate a Laplace
    smoothed frequency distribution of words, then uses this information
    to compute the information content of the lookup_word.
    """
    global N
    if N == 0:
        # poor man's lazy evaluation
        for sent in brown.sents():
            for word in sent:
                word = word.lower()
                if not (word in brown_freqs):
                    brown_freqs[word] = 0
                brown_freqs[word] = brown_freqs[word] + 1
                N = N + 1
    lookup_word = lookup_word.lower()
    n = 0 if not (lookup_word in brown_freqs) else brown_freqs[lookup_word]
    return 1.0 - (math.log(n + 1) / math.log(N + 1))

In [10]:
def semantic_vector(words, joint_words, info_content_norm):
    """
    Computes the semantic vector of a sentence. The sentence is passed in as
    a collection of words. The size of the semantic vector is the same as the
    size of the joint word set. The elements are 1 if a word in the sentence
    already exists in the joint word set, or the similarity of the word to the
    most similar word in the joint word set if it doesn't. Both values are 
    further normalized by the word's (and similar word's) information content
    if info_content_norm is True.
    """
    sent_set = set(words)
    semvec = np.zeros(len(joint_words))
    i = 0
    for joint_word in joint_words:
        if joint_word in sent_set:
            # if word in union exists in the sentence, s(i) = 1 (unnormalized)
            semvec[i] = 1.0
            if info_content_norm:
                semvec[i] = semvec[i] * math.pow(info_content(joint_word), 2)
        else:
            # find the most similar word in the joint set and set the sim value
            sim_word, max_sim = most_similar_word(joint_word, sent_set)
            semvec[i] = PHI if max_sim > PHI else 0.0
            if info_content_norm:
                semvec[i] = semvec[i] * info_content(joint_word) * info_content(sim_word)
        i = i + 1
    return semvec 

In [11]:
def semantic_similarity(sentence_1, sentence_2, info_content_norm):
    """
    Computes the semantic similarity between two sentences as the cosine
    similarity between the semantic vectors computed for each sentence.
    """
    words_1 = nltk.word_tokenize(sentence_1)
    words_2 = nltk.word_tokenize(sentence_2)
    joint_words = set(words_1).union(set(words_2))
    vec_1 = semantic_vector(words_1, joint_words, info_content_norm)
    vec_2 = semantic_vector(words_2, joint_words, info_content_norm)
    return np.dot(vec_1, vec_2.T) / (np.linalg.norm(vec_1) * np.linalg.norm(vec_2))


######################### word order similarity ##########################

In [12]:
def word_order_vector(words, joint_words, windex):
    """
    Computes the word order vector for a sentence. The sentence is passed
    in as a collection of words. The size of the word order vector is the
    same as the size of the joint word set. The elements of the word order
    vector are the position mapping (from the windex dictionary) of the 
    word in the joint set if the word exists in the sentence. If the word
    does not exist in the sentence, then the value of the element is the 
    position of the most similar word in the sentence as long as the similarity
    is above the threshold ETA.
    """
    wovec = np.zeros(len(joint_words))
    i = 0
    wordset = set(words)
    for joint_word in joint_words:
        if joint_word in wordset:
            # word in joint_words found in sentence, just populate the index
            wovec[i] = windex[joint_word]
        else:
            # word not in joint_words, find most similar word and populate
            # word_vector with the thresholded similarity
            sim_word, max_sim = most_similar_word(joint_word, wordset)
            if max_sim > ETA:
                wovec[i] = windex[sim_word]
            else:
                wovec[i] = 0
        i = i + 1
    return wovec

In [13]:
def word_order_similarity(sentence_1, sentence_2):
    """
    Computes the word-order similarity between two sentences as the normalized
    difference of word order between the two sentences.
    """
    words_1 = nltk.word_tokenize(sentence_1)
    words_2 = nltk.word_tokenize(sentence_2)
    joint_words = list(set(words_1).union(set(words_2)))
    windex = {x[1]: x[0] for x in enumerate(joint_words)}
    r1 = word_order_vector(words_1, joint_words, windex)
    r2 = word_order_vector(words_2, joint_words, windex)
    return 1.0 - (np.linalg.norm(r1 - r2) / np.linalg.norm(r1 + r2))


######################### overall similarity ##########################

In [14]:
def similarity(sentence_1, sentence_2, info_content_norm):
    """
    Calculate the semantic similarity between two sentences. The last 
    parameter is True or False depending on whether information content
    normalization is desired or not.
    """
    return DELTA * semantic_similarity(sentence_1, sentence_2, info_content_norm) + \
        (1.0 - DELTA) * word_order_similarity(sentence_1, sentence_2)

######################### main / test ##########################

In [21]:
# the results of the algorithm are largely dependent on the results of 
# the word similarities, so we should test this first...
word_pairs = [
  ["asylum", "fruit"],
  ["autograph", "shore"],
  ["autograph", "signature"],
  ["automobile", "car"],
  ["bird", "woodland"],
  ["boy", "rooster"],
  ["boy", "lad"],
  ["boy", "sage"],
  ["cemetery", "graveyard"],
  ["coast", "forest"],
  ["coast", "shore"],
  ["cock", "rooster"],
  ["cord", "smile"],
  ["cord", "string"],
  ["cushion", "pillow"],
  ["forest", "graveyard"],
  ["forest", "woodland"],
  ["furnace", "stove"],
  ["glass", "tumbler"],
  ["grin", "smile"],
  ["gem", "jewel"],
  ["hill", "woodland"],
  ["hill", "mound"],
  ["implement", "tool"],
  ["journey", "voyage"],
  ["magician", "oracle"],
  ["magician", "wizard"],
  ["midday", "noon"],
  ["oracle", "sage"],
  ["serf", "slave"],
  ["interest", "fee"],
  ["florida", "fl"]
]
    
for word_pair in word_pairs:
    print ("%12s\t%12s\t%.2f" % (word_pair[0], word_pair[1], word_similarity(word_pair[0], word_pair[1])))

      asylum	       fruit	0.30
   autograph	       shore	0.00
   autograph	   signature	0.82
  automobile	         car	1.00
        bird	    woodland	0.20
         boy	     rooster	0.11
         boy	         lad	0.82
         boy	        sage	0.37
    cemetery	   graveyard	1.00
       coast	      forest	0.36
       coast	       shore	0.80
        cock	     rooster	1.00
        cord	       smile	0.13
        cord	      string	0.82
     cushion	      pillow	0.82
      forest	   graveyard	0.20
      forest	    woodland	0.98
     furnace	       stove	0.17
       glass	     tumbler	0.82
        grin	       smile	0.99
         gem	       jewel	1.00
        hill	    woodland	0.36
        hill	       mound	0.99
   implement	        tool	0.82
     journey	      voyage	0.82
    magician	      oracle	0.30
    magician	      wizard	1.00
      midday	        noon	1.00
      oracle	        sage	0.37
        serf	       slave	0.55
    interest	         fee	0.82
     florida	          fl	1.00


In [24]:
sentence_pairs = [
    ["I like that bachelor.", "I like that unmarried man."],
    ["John is very nice.", "Is John very nice?"],
    ["Red alcoholic drink.", "A bottle of wine."],
    ["Red alcoholic drink.", "Fresh orange juice."],
    ["Red alcoholic drink.", "An English dictionary."],
    ["Red alcoholic drink.", "Fresh apple juice."],
    ["A glass of cider.", "A full cup of apple juice."],
    ["It is a dog.", "That must be your dog."],
    ["It is a dog.", "It is a log."],
    ["It is a dog.", "It is a pig."],
    ["Dogs are animals.", "They are common pets."],
    ["Canis familiaris are animals.", "Dogs are common pets."],
    ["I have a pen.", "Where do you live?"],
    ["I have a pen.", "Where is ink?"],
    ["I have a hammer.", "Take some nails."],
    ["I have a hammer.", "Take some apples."]
]
for sent_pair in sentence_pairs:
    print ("%30s\t%30s\t%.3f" % (sent_pair[0], sent_pair[1],
        similarity(sent_pair[0], sent_pair[1], False)))

         I like that bachelor.	    I like that unmarried man.	0.804
            John is very nice.	            Is John very nice?	0.580
          Red alcoholic drink.	             A bottle of wine.	0.430
          Red alcoholic drink.	           Fresh orange juice.	0.451
          Red alcoholic drink.	        An English dictionary.	0.284
          Red alcoholic drink.	            Fresh apple juice.	0.395
             A glass of cider.	    A full cup of apple juice.	0.662
                  It is a dog.	        That must be your dog.	0.481
                  It is a dog.	                  It is a log.	0.860
                  It is a dog.	                  It is a pig.	0.878
             Dogs are animals.	         They are common pets.	0.570
 Canis familiaris are animals.	         Dogs are common pets.	0.501
                 I have a pen.	            Where do you live?	0.101
                 I have a pen.	                 Where is ink?	0.139
              I have a hammer.	              Tak

In [26]:
sentence_pairs = [
    ["you can enjoy a lower interest rate on new eligible charges added to your Pay Over Time feature for limited time."],
    ["you have a pay over time fature on your account that gives you the option to pay certain charges over time with interest"],
    ["if you are enrolled in pay over time select, this promotional rate will also apply to eligibile purchase added at your request. "],
]

query_sentence = "eligible rate will be applied to purchase when you enrolled to pay over time select."

for sent_pair in sentence_pairs:
    print ("%s\nQuery: %s\nSimilarity: %.3f\n" % (sent_pair[0], query_sentence, 
        similarity(sent_pair[0], query_sentence, False)))

you can enjoy a lower interest rate on new eligible charges added to your Pay Over Time feature for limited time.
Query: eligible rate will be applied to purchase when you enrolled to pay over time select.
Similarity: 0.506

you have a pay over time fature on your account that gives you the option to pay certain charges over time with interest
Query: eligible rate will be applied to purchase when you enrolled to pay over time select.
Similarity: 0.460

if you are enrolled in pay over time select, this promotional rate will also apply to eligibile purchase added at your request. 
Query: eligible rate will be applied to purchase when you enrolled to pay over time select.
Similarity: 0.625

